In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/솔룩스 데이텀/meloncrawling.csv")
df

,0,1,2,3
0,2010,Bad Girl Good Girl,미쓰에이,You don't know me\n\nYou don't know me\n\nYou ...
1,2010,잔소리 (With 2AM 슬옹),아이유,늦게 다니지좀 마\n\n술은 멀리좀 해봐\n\n열살짜리 애처럼 말을 안듣니\n\n정...
2,2010,죽을 만큼 아파서 (Feat. 멜로우),MC몽,I found the way to\nlet you leave\nI never rea...
3,2010,못해 (Feat. 美),포맨 (4MEN),이제는 밥을 먹어도\n눈물없이는 삼키지 못해\n억지로 먹고 먹어도 속이 늘 허전해\...
4,2010,죽어도 못 보내,2am,어려도 아픈건 똑같아\n세상을 잘 모른다고\n아픈걸 모르진 않아\n괜찮아 질거라고 ...
...,...,...,...,...
1176,2021,함께 했는데 이별은 나 혼자인 거야,이소정,이젠 누군갈 만나는 게 난\n아직은 힘이 들것 같아\n다시는 믿고 싶지 않아\n왜 ...
1177,2021,비가 오는 날엔 (2021),헤이즈 (Heize),세상이 어두워지고\n조용히 비가 내리면\n여전히 그대로\n오늘도 어김없이 난\n벗어...
1178,2021,살다가,SG 워너비,살아도 사는 게 아니래\n너 없는 하늘에\n창 없는 감옥 같아서\n웃어도 웃는 게 ...
1179,2021,별빛 같은 나의 사랑아,임영웅,당신이 얼마나 내게\n소중한 사람인지\n세월이 흐르고 보니\n이제 알 것 같아요\n...


가사에서 영어 제거

In [ ]:
import re

# 한글과 띄어쓰기만 남기고 제거해주는 함수 test
def test(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return(result)

한글 가사 토큰화

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 48.3 MB/s 


In [ ]:
from konlpy.tag import Kkma, Okt

In [ ]:
okt = Okt()

In [ ]:
def tokenization(korean_lyrics):
  lyric_final = [] # 토큰화된 최종 가사를 저장할 리스트
  for lyric in korean_lyrics:
    stem_pos_results = okt.pos(lyric, stem =True)

    lst = []
    for a, b in stem_pos_results:
      if b == 'Noun' or b == 'VerbPrefix' or b == 'Adjective':
        lst.append(a) # 해당되는 품사의 단어만 리스트에 저장
    lyric_final += lst
  return lyric_final

In [ ]:
final = []

In [ ]:
for n in tqdm(range(len(df['3']))): #len(df['3']) = 1181
  sentence = df['3'][n]
  sentences = sentence.splitlines()

  korean_lyrics = []

  for sent in sentences:
    korean_lyric = test(sent)
    korean_lyric = korean_lyric.strip()
    if korean_lyric == '':
      continue
    korean_lyrics.append(korean_lyric) # korean_lyrics 리스트에 한글 가사만 저장

  make_set = set(korean_lyrics)
  korean_lyrics = list(make_set) # 중복되는 부분 제거
  
  final.append(tokenization(korean_lyrics)) # 토큰화하여 final 리스트에 저장

100%|██████████| 1181/1181 [00:48<00:00, 24.16it/s]


제목, 가수, 토큰화 결과 합치기

In [ ]:
df_song = df['1']
df_song

0                    Bad Girl Good Girl
1                     잔소리 (With 2AM 슬옹)
2                 죽을 만큼 아파서 (Feat. 멜로우)
3                          못해 (Feat. 美)
4                              죽어도 못 보내
                     ...               
1176                함께 했는데 이별은 나 혼자인 거야
1177                    비가 오는 날엔 (2021)
1178                                살다가
1179                       별빛 같은 나의 사랑아
1180    CREDIT (Feat. 염따, 기리보이, Zion.T)
Name: 1, Length: 1181, dtype: object

In [ ]:
df_singer = df['2']
df_singer

0               미쓰에이
1                아이유
2                MC몽
3          포맨 (4MEN)
4                2am
            ...     
1176             이소정
1177     헤이즈 (Heize)
1178          SG 워너비
1179             임영웅
1180    릴보이 (lIlBOI)
Name: 2, Length: 1181, dtype: object

In [ ]:
df_song_info = df['1'].str.cat(df['2'], sep=' - ')
print(df_song_info)

0                            Bad Girl Good Girl - 미쓰에이
1                              잔소리 (With 2AM 슬옹) - 아이유
2                          죽을 만큼 아파서 (Feat. 멜로우) - MC몽
3                             못해 (Feat. 美) - 포맨 (4MEN)
4                                       죽어도 못 보내 - 2am
                             ...                      
1176                         함께 했는데 이별은 나 혼자인 거야 - 이소정
1177                     비가 오는 날엔 (2021) - 헤이즈 (Heize)
1178                                      살다가 - SG 워너비
1179                                별빛 같은 나의 사랑아 - 임영웅
1180    CREDIT (Feat. 염따, 기리보이, Zion.T) - 릴보이 (lIlBOI)
Name: 1, Length: 1181, dtype: object


In [ ]:
melon_token = zip(df_song_info, final)

In [ ]:
melon = pd.DataFrame(melon_token)
melon

,0,1
0,Bad Girl Good Girl - 미쓰에이,"[속, 왜, 자꾸, 자신, 없다, 저, 뒤, 뒤, 뒤, 내, 겉모습, 그, 위선, ..."
1,잔소리 (With 2AM 슬옹) - 아이유,"[이렇다, 자꾸, 더, 못, 참고, 화가, 소리, 하나, 열, 널, 위, 소리, 니..."
2,죽을 만큼 아파서 (Feat. 멜로우) - MC몽,"[새, 내, 눈물, 내, 말좀, 잠깐, 죽, 것, 같다, 애, 전화, 좀, 분만, ..."
3,못해 (Feat. 美) - 포맨 (4MEN),"[날, 날, 것, 못, 너, 난, 못, 억지로, 늘, 허전하다, 눈물, 없다, 못,..."
4,죽어도 못 보내 - 2am,"[괜찮다, 왜, 거짓말, 해, 니, 날, 아프다, 걸, 정말, 거짓말, 해, 내, ..."
...,...,...
1176,함께 했는데 이별은 나 혼자인 거야 - 이소정,"[그렇다, 내, 널, 홀로, 또, 미안하다, 날, 사랑, 그땐, 내, 곁, 있다, ..."
1177,비가 오는 날엔 (2021) - 헤이즈 (Heize),"[다시, 널, 이제, 아니다, 걸, 알, 그렇다, 건, 밤, 것, 같다, 애, 너,..."
1178,살다가 - SG 워너비,"[너, 없다, 하늘, 나, 잠, 순간, 창, 없다, 감옥, 같다, 내, 곁, 있다,..."
1179,별빛 같은 나의 사랑아 - 임영웅,"[고맙다, 행복하다, 소중하다, 사람, 밤하늘, 당신, 나, 영원하다, 사랑, 사랑..."


In [ ]:
from google.colab import files
melon.to_csv("meloncrawling_tokenization.csv", encoding='utf-8-sig', index=False)
files.download('meloncrawling_tokenization.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>